In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, f1_score

In [ ]:
df_train = pd.read_csv('df_train_clean.csv')
df_dev = pd.read_csv('df_dev_clean.csv')
df_test = pd.read_csv('df_test_clean.csv')

In [ ]:
df_train.head()

,Unnamed: 0,content,Constructiveness,Toxicity,Title,Topic,content_clean
0,6326,Thật tuyệt vời...!!!,0,0,Những 'bước tiến diệu kỳ' của Trúc Nhi - Diệu Nhi,SucKhoe,Thật tuyệt vời...!!!
1,7835,"mỹ đã tuột dốc quá nhiều rồi, giờ muốn vực dậy...",1,0,Hình tượng Mỹ sụp đổ trong lòng người dân thế ...,TheGioi,"mỹ đã tuột dốc quá nhiều rồi, giờ muốn vực dậy..."
2,4690,tôi thấy người lái xe hơi bấm còi mới là người...,1,1,Cả trăm người đạp xe thể dục bịt kín đường,OtoXemay,tôi thấy người lái xe hơi bấm còi mới là người...
3,6011,Coi dịch là giặc. Đã mang tên đó mà xâm nhập V...,0,0,11 ngày không lây nhiễm nCoV cộng đồng,SucKhoe,Coi dịch là giặc. Đã mang tên đó mà xâm nhập v...
4,9303,Thương các bé quá! Các con còn quá nhỏ mà đã p...,0,0,5 trẻ chết đuối dưới ao,ThoiSu,Thương các bé quá! Các con còn quá nhỏ mà đã p...


In [ ]:
col_text = 'content_clean'
col_label = 'Toxicity'

In [ ]:
X_train_text = df_train[col_text]
y_train = df_train[col_label]
X_dev_text = df_dev[col_text]
y_dev = df_dev[col_label]
X_test_text = df_test[col_text]
y_test = df_test[col_label]

In [ ]:
# --- TF-IDF & Bag of Words ---
vectorizer_options = {
    'tfidf': TfidfVectorizer(),
    'bow': CountVectorizer()
}

# --- n-gram & feature ---
vectorizer_param_grid = {
    'max_features': [3000, 5000, 7000],
    'ngram_range': [(1,3), (1,7)],  # unigram, unigram+bigram
}

In [ ]:
# --- Finetune SVM & Logistic Regression ---
model_param_grid = {
    'svm': {
        'C': [0.1, 1, 10],
        'penalty': ['l2'],
        'loss': ['hinge', 'squared_hinge'],
        'max_iter': [1000, 2000]
    },
    'logistic': {
        'C': [0.1, 1, 10],
        'penalty': ['l2'],
        'solver': ['lbfgs', 'saga'],
        'max_iter': [200, 500]
    }
}

In [ ]:
def finetune_and_report(vec_name, vectorizer, model_name, model, model_params):
    print(f"\n========== {model_name.upper()} với {vec_name.upper()} ==========")
    # grid vectorizer + model
    param_grid = {**{f'vectorizer__{k}': v for k, v in vectorizer_param_grid.items()},
                  **{f'model__{k}': v for k, v in model_params.items()}}
    from sklearn.pipeline import Pipeline

    pipe = Pipeline([
        ('vectorizer', vectorizer),
        ('model', model)
    ])
    grid = GridSearchCV(pipe, param_grid, scoring='f1_macro', cv=3, n_jobs=-1, verbose=1)
    grid.fit(X_train_text, y_train)

    print(f"Best parameters: {grid.best_params_}")
    print(f"Best crossval F1_macro: {grid.best_score_:.4f}")

    # classification report
    y_pred_dev = grid.predict(X_dev_text)
    f1_dev = f1_score(y_dev, y_pred_dev, average='macro')
    print("Dev F1 Macro:", f1_dev)
    print("Classification report trên dev:")
    print(classification_report(y_dev, y_pred_dev, digits=4))
    return grid

In [ ]:
best_models = {}
for vec_name, vectorizer in vectorizer_options.items():
    # SVM
    best_models[f'svm_{vec_name}'] = finetune_and_report(
        vec_name, vectorizer, 'svm', LinearSVC(), model_param_grid['svm']
    )
    # Logistic Regression
    best_models[f'logistic_{vec_name}'] = finetune_and_report(
        vec_name, vectorizer, 'logistic', LogisticRegression(), model_param_grid['logistic']
    )


========== SVM với TFIDF ==========
Fitting 3 folds for each of 72 candidates, totalling 216 fits
Best parameters: {'model__C': 1, 'model__loss': 'squared_hinge', 'model__max_iter': 1000, 'model__penalty': 'l2', 'vectorizer__max_features': 7000, 'vectorizer__ngram_range': (1, 3)}
Best crossval F1_macro: 0.6505
Dev F1 Macro: 0.6380345439521878
Classification report trên dev:
              precision    recall  f1-score   support

           0     0.9073    0.9745    0.9397      1768
           1     0.5545    0.2414    0.3363       232

    accuracy                         0.8895      2000
   macro avg     0.7309    0.6080    0.6380      2000
weighted avg     0.8664    0.8895    0.8697      2000


========== LOGISTIC với TFIDF ==========
Fitting 3 folds for each of 72 candidates, totalling 216 fits
Best parameters: {'model__C': 10, 'model__max_iter': 200, 'model__penalty': 'l2', 'model__solver': 'lbfgs', 'vectorizer__max_features': 7000, 'vectorizer__ngram_range': (1, 7)}
Best crossval 

In [ ]:
for name, model in best_models.items():
    y_pred_test = model.predict(X_test_text)
    f1 = f1_score(y_test, y_pred_test, average='macro')
    print(f"\n== {name.upper()} trên TEST ==")
    print("F1 Macro:", f1)
    print("Classification report trên test:")
    print(classification_report(y_test, y_pred_test, digits=4))


== SVM_TFIDF trên TEST ==
F1 Macro: 0.6635765550239234
Classification report trên test:
              precision    recall  f1-score   support

           0     0.9186    0.9640    0.9408       890
           1     0.5152    0.3091    0.3864       110

    accuracy                         0.8920      1000
   macro avg     0.7169    0.6366    0.6636      1000
weighted avg     0.8742    0.8920    0.8798      1000


== LOGISTIC_TFIDF trên TEST ==
F1 Macro: 0.6592735454837673
Classification report trên test:
              precision    recall  f1-score   support

           0     0.9170    0.9685    0.9421       890
           1     0.5333    0.2909    0.3765       110

    accuracy                         0.8940      1000
   macro avg     0.7252    0.6297    0.6593      1000
weighted avg     0.8748    0.8940    0.8799      1000


== SVM_BOW trên TEST ==
F1 Macro: 0.6496641994081561
Classification report trên test:
              precision    recall  f1-score   support

           0     0.92